# Main homework

> Автор: Никита Разуваев

In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F

import os
import glob


sc = pyspark.SparkContext(appName="lsml-app-mhw")
se = SparkSession(sc)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-03-28 16:27:26,587 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
2022-03-28 16:27:40,507 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
2022-03-28 16:27:40,520 WARN cluster.YarnSchedulerBackend$YarnSchedulerEndpo

In [2]:
%cd /home/ubuntu/storage/outbrain

/home/ubuntu/storage/outbrain


In [3]:
! hdfs dfs -ls /user/outbrain/data

Found 9 items
-rw-r--r--   1 ubuntu hadoop   506953370 2022-03-16 19:51 /user/outbrain/data/clicks_test.csv
-rw-r--r--   1 ubuntu hadoop  1486734654 2022-03-16 19:43 /user/outbrain/data/clicks_train.csv
-rw-r--r--   1 ubuntu hadoop   118017029 2022-03-16 19:42 /user/outbrain/data/documents_categories.csv
-rw-r--r--   1 ubuntu hadoop   324096832 2022-03-16 19:45 /user/outbrain/data/documents_entities.csv
-rw-r--r--   1 ubuntu hadoop    89380566 2022-03-17 18:52 /user/outbrain/data/documents_meta.csv
-rw-r--r--   1 ubuntu hadoop   339473038 2022-03-16 19:51 /user/outbrain/data/documents_topics.csv
-rw-r--r--   1 ubuntu hadoop  1208549589 2022-03-16 19:41 /user/outbrain/data/events.csv
-rw-r--r--   1 ubuntu hadoop 94931571958 2022-03-17 16:35 /user/outbrain/data/page_views.csv
-rw-r--r--   1 ubuntu hadoop    13886609 2022-03-16 19:43 /user/outbrain/data/promoted_content.csv


In [4]:
from subprocess import Popen, PIPE


hdfs_path = '/user/outbrain/data'
process = Popen(f'hdfs dfs -ls -h {hdfs_path}', shell=True, stdout=PIPE, stderr=PIPE)
std_out, std_err = process.communicate()
file_list = [fn.split(' ')[-1] for fn in std_out.decode().split('\n')[1:]][:-1]
file_list

['/user/outbrain/data/clicks_test.csv',
 '/user/outbrain/data/clicks_train.csv',
 '/user/outbrain/data/documents_categories.csv',
 '/user/outbrain/data/documents_entities.csv',
 '/user/outbrain/data/documents_meta.csv',
 '/user/outbrain/data/documents_topics.csv',
 '/user/outbrain/data/events.csv',
 '/user/outbrain/data/page_views.csv',
 '/user/outbrain/data/promoted_content.csv']

## Перегоним данные в parquet

In [ ]:
data_csv = {
    fp.split('/')[-1].split('.')[0]:
        se.read.option("mode", "DROPMALFORMED")
        .option('sep', ",")
        .csv(fp, header=True, inferSchema=True)
        for fp in file_list
}

In [ ]:
for tbl in data_csv:
    print(f'Saving {tbl} to parquet...')
    data_csv[tbl].write.parquet(f"/user/outbrain/parquet/{tbl}.parquet")

In [ ]:
! hdfs dfs -ls /user/outbrain/parquet

## Основная часть

In [5]:
import string
import re


def to_number(raw_value):
    try:
        return float(raw_value)
    except:
        return 0.0

se.udf.register("to_number", to_number, "float")

<function __main__.to_number(raw_value)>

In [38]:
data = {
    fp.split('/')[-1].split('.')[0]:
        se.read.parquet(
            '/user/outbrain/parquet/{}.parquet'
            .format(fp.split('/')[-1].split('.')[0])
        )
        for fp in file_list
}
for tbl in data:
    data[tbl].registerTempTable(tbl)
data

{'clicks_test': DataFrame[display_id: int, ad_id: int],
 'clicks_train': DataFrame[display_id: int, ad_id: int, clicked: int],
 'documents_categories': DataFrame[document_id: int, category_id: int, confidence_level: double],
 'documents_entities': DataFrame[document_id: int, entity_id: string, confidence_level: double],
 'documents_meta': DataFrame[document_id: int, source_id: int, publisher_id: int, publish_time: string],
 'documents_topics': DataFrame[document_id: int, topic_id: int, confidence_level: double],
 'events': DataFrame[display_id: int, uuid: string, document_id: int, timestamp: int, platform: string, geo_location: string],
 'page_views': DataFrame[uuid: string, document_id: int, timestamp: int, platform: int, geo_location: string, traffic_source: int],
 'promoted_content': DataFrame[ad_id: int, document_id: int, campaign_id: int, advertiser_id: int]}

- Основная идея: берем бейслайн из ссылки в домашке и накручиваем еще фичей из оставшихся таблиц

In [39]:
data['documents_categories'] = data['documents_categories'].groupby(['document_id'])\
.agg(F.array_join(F.collect_list('category_id'), ' ').alias('category_id'))
data['documents_categories'].limit(10).toPandas()

,document_id,category_id
0,148,1403 1702
1,463,1513 1808
2,471,1504 1609
3,496,1210 1203
4,833,1305 2004
5,1088,2006 1210
6,1238,1100 1407
7,1342,1408 2004
8,1580,1403 1402
9,1591,1608 1603


In [40]:
data['documents_entities'] = data['documents_entities'].groupby(['document_id'])\
.agg(F.array_join(F.collect_list(F.hash('entity_id')), ' ').alias('entity_id'))
data['documents_entities'].limit(10).toPandas()

,document_id,entity_id
0,148,1119203076 128006713 -75275561 -577457920 -101...
1,463,1384943934
2,496,1119762379
3,833,1750353926 -1688486307
4,1088,1704337977
5,1580,449082193 -416459619 1231395558 1328932179
6,1645,-2036804371 1796593387 -1803384746 1295841012 ...
7,1959,1028614374
8,2122,168792357 -1014031353 944140881 1207588968 150...
9,2142,-287093450 -2024458271 -1249415393 -1141032337...


In [41]:
data['documents_meta'] = data['documents_meta']\
.withColumn('dayofweek', F.dayofweek('publish_time'))\
.withColumn('dayofmonth', F.dayofmonth('publish_time'))\
.drop('publish_time')
data['documents_meta'].limit(10).toPandas()

,document_id,source_id,publisher_id,dayofweek,dayofmonth
0,788428,579,9,2,16
1,864787,579,9,2,22
2,817971,579,9,2,13
3,806663,579,9,2,1
4,796001,579,9,6,3
5,799992,579,9,5,23
6,863420,579,9,2,21
7,814101,579,9,6,7
8,789170,579,9,2,9
9,831902,579,9,2,24


In [42]:
data['documents_topics'] = data['documents_topics'].groupby(['document_id'])\
.agg(F.array_join(F.collect_list('topic_id'), ' ').alias('topic_id'))
data['documents_topics'].limit(10).toPandas()

,document_id,topic_id
0,148,153 140 8 172 244 179 36 2 64 10 216
1,463,181 292 24 254 167
2,471,285 238 153 193
3,496,244 294 196 102 20 264 26
4,833,294 89 174 86 264 279 231
5,1088,107 75 153 64 94
6,1238,89 221 192 236
7,1342,271 283 181 225 198 234
8,1580,8 37 136 12 160 130
9,1591,260


In [43]:
data['promoted_content'] = data['promoted_content']\
.join(data['documents_categories'],
      on='document_id', how='left')\
.join(data['documents_entities'],
      on='document_id', how='left')\
.join(data['documents_meta'],
      on='document_id', how='left')\
.join(data['documents_topics'],
      on='document_id', how='left')\
.withColumnRenamed("document_id","ad_document_id")\
.withColumnRenamed("category_id","ad_category_id")\
.withColumnRenamed("entity_id","ad_entity_id")\
.withColumnRenamed("source_id","ad_source_id")\
.withColumnRenamed("publisher_id","ad_publisher_id")\
.withColumnRenamed("dayofweek","ad_dayofweek")\
.withColumnRenamed("dayofmonth","ad_dayofmonth")\
.withColumnRenamed("topic_id","ad_topic_id")\
.fillna(-1).fillna('0')
data['promoted_content'].limit(10).toPandas()

,ad_document_id,ad_id,campaign_id,advertiser_id,ad_category_id,ad_entity_id,ad_source_id,ad_publisher_id,ad_dayofweek,ad_dayofmonth,ad_topic_id
0,29834,45,26,39,1513 1514,0,2586,278,3,28,256
1,29834,1576,292,39,1513 1514,0,2586,278,3,28,256
2,29894,92,26,39,1502 1510,0,2586,278,-1,-1,0
3,29993,1459,297,39,1514 1513,0,2586,278,-1,-1,0
4,57039,306,53,86,1909 1406,1518997114 -672320419 -5553965,333,318,5,29,203 16 223 209 238
5,76885,83605,9136,106,1805 1513,0,4974,363,-1,-1,292
6,90461,240962,25864,128,2004 1100,-2024458271,31,535,5,13,271 95 249 43 2 71 66 1 232 203
7,130003,2576,127,138,2100 1403,0,3045,310,2,22,16 125
8,131931,645,53,86,1505 1403,-1797832892 -1980372975,333,318,6,5,272 195
9,132406,650,99,76,1511 1503,0,3183,139,-1,-1,0


In [55]:
events_f = se.sql("""
    SELECT 
        display_id, document_id,
        int(to_number(platform)) as platform_id,
        hash(uuid) as uuid_id,
        hash(geo_location) as geo_location_id,
        hash(split(geo_location, '>')[0]) as country,
        hash(split(geo_location, '>')[1]) as state,
        int(to_number(split(geo_location, '>')[2])) as dma,
        int((timestamp / 1000) / 60 / 60 / 24) as day
    FROM events
""")\
.join(data['documents_categories'],
      on='document_id', how='left')\
.join(data['documents_entities'],
      on='document_id', how='left')\
.join(data['documents_meta'],
      on='document_id', how='left')\
.join(data['documents_topics'],
      on='document_id', how='left')\
.fillna(-1).fillna('0')

In [47]:
events_f.limit(10).toPandas()

,document_id,display_id,platform_id,uuid_id,geo_location_id,country,state,dma,day,category_id,entity_id,source_id,publisher_id,dayofweek,dayofmonth,topic_id
0,148,4006613,2,2075984812,-716265764,-1077041708,-1254539730,518,2,1403 1702,1119203076 128006713 -75275561 -577457920 -101...,1787,118,3,1,153 140 8 172 244 179 36 2 64 10 216
1,148,7007148,1,1211871736,2129082374,-483336299,-1301460872,0,5,1403 1702,1119203076 128006713 -75275561 -577457920 -101...,1787,118,3,1,153 140 8 172 244 179 36 2 64 10 216
2,148,7022830,1,-1297441364,-1206119460,1062621904,1297677906,0,5,1403 1702,1119203076 128006713 -75275561 -577457920 -101...,1787,118,3,1,153 140 8 172 244 179 36 2 64 10 216
3,148,12129180,2,-1300864767,320090763,1193196075,1573909955,0,9,1403 1702,1119203076 128006713 -75275561 -577457920 -101...,1787,118,3,1,153 140 8 172 244 179 36 2 64 10 216
4,148,15754500,2,-1069099532,-1621535015,-1077041708,-916423235,524,12,1403 1702,1119203076 128006713 -75275561 -577457920 -101...,1787,118,3,1,153 140 8 172 244 179 36 2 64 10 216
5,148,16336027,1,766161825,1318500090,-1077041708,-912215920,602,12,1403 1702,1119203076 128006713 -75275561 -577457920 -101...,1787,118,3,1,153 140 8 172 244 179 36 2 64 10 216
6,496,3381251,1,1682577209,1611187875,-1077041708,-1832830425,539,2,1210 1203,1119762379,2186,184,2,29,244 294 196 102 20 264 26
7,1342,8020113,1,2005259149,394423548,-912215920,1673531880,0,6,1408 2004,0,475,937,5,1,271 283 181 225 198 234
8,1342,18473607,1,541097676,-757005368,899366864,1673531880,0,7,1408 2004,0,475,937,5,1,271 283 181 225 198 234
9,1342,12346919,1,1045211388,-931697849,-1077041708,1517794078,609,9,1408 2004,0,475,937,5,1,271 283 181 225 198 234


- Собираем все в трейн и тест

In [48]:
train_features = data['clicks_train'].\
join(events_f, on='display_id').\
join(data['promoted_content'], on='ad_id')\
.fillna('0').cache()

test_features = data['clicks_test'].\
join(events_f, on='display_id').\
join(data['promoted_content'], on='ad_id')\
.fillna('0').cache()

2022-03-28 16:58:00,757 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [49]:
train_features.limit(10).toPandas()

,ad_id,display_id,clicked,document_id,platform_id,uuid_id,geo_location_id,country,state,dma,...,ad_document_id,campaign_id,advertiser_id,ad_category_id,ad_entity_id,ad_source_id,ad_publisher_id,ad_dayofweek,ad_dayofmonth,ad_topic_id
0,148,3678937,0,1834297,1,-349639493,1318500090,-1077041708,-912215920,602,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
1,148,3262691,0,1773858,1,-2059493111,354655130,-1077041708,410653915,508,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
2,148,3267986,0,348758,1,87621201,-441949987,-1077041708,1886535265,803,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
3,148,3783605,0,846331,1,1825929414,531595220,-1077041708,1054775907,0,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
4,148,8944408,0,865916,1,995847873,-629717874,-1077041708,-1034888170,820,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
5,148,8869255,0,60548,1,-1068804253,-1141187544,-1077041708,2107043530,538,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
6,148,3195987,0,1773858,1,-523623322,1318500090,-1077041708,-912215920,602,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
7,148,4349019,0,693594,1,2127407726,550529026,-1077041708,1854557654,505,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
8,148,9944564,0,850459,1,-1680123944,1241665635,-1077041708,1054775907,510,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
9,471,10165719,0,1354341,3,-706342077,-25224295,-1077041708,1054775907,547,...,76819,59,76,1606 1513,0,3183,139,-1,-1,0


In [50]:
test_features.limit(10).toPandas()

,ad_id,display_id,document_id,platform_id,uuid_id,geo_location_id,country,state,dma,day,...,ad_document_id,campaign_id,advertiser_id,ad_category_id,ad_entity_id,ad_source_id,ad_publisher_id,ad_dayofweek,ad_dayofmonth,ad_topic_id
0,148,17454023,821539,1,-108603948,1532322040,-1077041708,1054775907,535,2,...,471520,34,19,1515 1512,0,2644,113,5,5,93 285 296 292
1,463,22875888,1361813,3,-1791857317,-1273146981,-1077041708,1517794078,616,14,...,577867,68,19,1511 1512,-1739988653,1106,113,-1,-1,20 285 238 247 9 112
2,463,22712434,1002800,1,1356160377,1318500090,-1077041708,-912215920,602,14,...,577867,68,19,1511 1512,-1739988653,1106,113,-1,-1,20 285 238 247 9 112
3,463,21673772,272944,1,1135079619,-1913106605,-1077041708,1886535265,807,14,...,577867,68,19,1511 1512,-1739988653,1106,113,-1,-1,20 285 238 247 9 112
4,471,18671573,315152,1,-154083689,862934729,-1077041708,-654501249,623,7,...,76819,59,76,1606 1513,0,3183,139,-1,-1,0
5,471,18636483,893701,3,2042208444,-1835090850,-1077041708,410653915,566,7,...,76819,59,76,1606 1513,0,3183,139,-1,-1,0
6,471,18598685,1155255,1,1715450529,1187335289,-1077041708,1886535265,800,7,...,76819,59,76,1606 1513,0,3183,139,-1,-1,0
7,1238,17504439,670860,2,552705610,-1560381294,-911176333,40355443,0,2,...,185913,114,67,2006 2002,1312092024 -1434132067,2633,-1,5,12,75 143 16 147 102
8,1238,17510370,671762,2,-746775905,-1452971272,-911176333,-1613909223,0,2,...,185913,114,67,2006 2002,1312092024 -1434132067,2633,-1,5,12,75 143 16 147 102
9,1238,16976050,549825,2,-1127814001,-2092950637,-911176333,113559898,0,0,...,185913,114,67,2006 2002,1312092024 -1434132067,2633,-1,5,12,75 143 16 147 102


In [51]:
! hdfs dfs -rm -r /user/outbrain/parquet/train_features.parquet
! hdfs dfs -rm -r /user/outbrain/parquet/test_features.parquet
train_features.write.parquet("/user/outbrain/parquet/train_features.parquet")
test_features.write.parquet("/user/outbrain/parquet/test_features.parquet")

rm: `/user/outbrain/parquet/train_features.parquet': No such file or directory
rm: `/user/outbrain/parquet/test_features.parquet': No such file or directory


In [60]:
features_to_rm = [
    'display_id', 'day', 'uuid_id', 'clicked', 
    # 'ad_confidence_level', 'confidence_level' # numeric
]
# float_space = ['ad_confidence_level', 'confidence_level']
feature_space = train_features.columns
for feat in features_to_rm:
    feature_space.remove(feat)
feature_space # , float_space

['ad_id',
 'document_id',
 'platform_id',
 'geo_location_id',
 'country',
 'state',
 'dma',
 'category_id',
 'entity_id',
 'source_id',
 'publisher_id',
 'dayofweek',
 'dayofmonth',
 'topic_id',
 'ad_document_id',
 'campaign_id',
 'advertiser_id',
 'ad_category_id',
 'ad_entity_id',
 'ad_source_id',
 'ad_publisher_id',
 'ad_dayofweek',
 'ad_dayofmonth',
 'ad_topic_id']

In [29]:
distr_of_days = train_features.select('day').groupby('day').count().toPandas()
distr_of_days['percentage'] = distr_of_days['count'] / distr_of_days['count'].sum() * 100
distr_of_days

,day,count,percentage
0,12,6443253,7.393992
1,1,6660916,7.643773
2,6,7477566,8.580924
3,3,6508676,7.469069
4,5,5708919,6.551303
5,9,7120254,8.170889
6,4,5678354,6.516228
7,8,7141971,8.195810
8,7,7041148,8.080110
9,10,6975398,8.004659


In [61]:
train_data, val_data = train_features.randomSplit([.9, .1], 20032022)
train_data, val_data = train_data.cache(), val_data.cache()
'''
train_data = train_features.where(F.col('day') < 11).cache()
val_data = train_features.where(F.col('day') >= 11).cache()
'''
test_data = test_features

In [62]:
import string

def convert_to_vw(
    data,
    feature_space=None,
    float_space=None,
    task='train',
    target_name='clicked',
    tag_names=None,
    importance=''
):
    data = data.asDict()
    
    if task == 'train':
        target_name = target_name
        if data[target_name] == 0:
            data[target_name] = -1
            importance = str(1) + ' ' if type(importance) == (int or float) else ''
        else:
            importance = str(importance) + ' ' if type(importance) == (int or float) else ''
    else:
        target_name = ''
        data[target_name] = ''
    
    aliases = string.ascii_letters[:len(feature_space)]
    if feature_space:
        features = ''.join([
            f'|{aliases[i]} {data[feat]} '
            for i, feat in enumerate(feature_space)
        ])
    else:
        features = ''.join([
            f'|{aliases[i]} {v} '
            for i, (k, v) in enumerate(data.items())
            if k != target_name
        ])
    if float_space:
        features += '|float_values ' + ' '.join([
            f'{feat}:{data[feat]}'
            for feat in float_space
        ])
    
    if tag_names:
        tags = ''.join([f'{tag_name}_' + str(data[tag_name]) + '_' for tag_name in tag_names])[:-1]
    else:
        tags = ''
    
    template = '{target} {importance}{tags}{features}'\
        .format(target=data[target_name], importance=importance, features=features, tags=tags)
    return template.strip()

for i in train_data.take(5):
    print(
        convert_to_vw(i, task='train', # importance=2,
                      feature_space=feature_space,
                      # float_space=float_space,
                      tag_names=['display_id', 'ad_id'])
    )

-1 display_id_3195987_ad_id_148|a 148 |b 1773858 |c 1 |d 1318500090 |e -1077041708 |f -912215920 |g 602 |h 1515 1506 |i -564571673 -2087840067 |j 872 |k 26 |l -1 |m -1 |n 93 285 276 174 250 117 |o 471520 |p 34 |q 19 |r 1515 1512 |s 0 |t 2644 |u 113 |v 5 |w 5 |x 93 285 296 292
-1 display_id_3262691_ad_id_148|a 148 |b 1773858 |c 1 |d 354655130 |e -1077041708 |f 410653915 |g 508 |h 1515 1506 |i -564571673 -2087840067 |j 872 |k 26 |l -1 |m -1 |n 93 285 276 174 250 117 |o 471520 |p 34 |q 19 |r 1515 1512 |s 0 |t 2644 |u 113 |v 5 |w 5 |x 93 285 296 292
-1 display_id_3267986_ad_id_148|a 148 |b 348758 |c 1 |d -441949987 |e -1077041708 |f 1886535265 |g 803 |h 1515 1510 |i -1475663123 -2027717077 |j 2760 |k 939 |l -1 |m -1 |n 294 117 129 296 62 244 291 285 170 163 |o 471520 |p 34 |q 19 |r 1515 1512 |s 0 |t 2644 |u 113 |v 5 |w 5 |x 93 285 296 292
-1 display_id_3678937_ad_id_148|a 148 |b 1834297 |c 1 |d 1318500090 |e -1077041708 |f -912215920 |g 602 |h 1515 1612 |i -1120691283 1939100908 |j 6862 |k

In [63]:
fname = '/user/outbrain/vw/train.vw' # _randomSplit
! hdfs dfs -rm -r $fname
train_data.rdd\
.map(lambda x: convert_to_vw(
    x, feature_space=feature_space, # float_space=float_space, 
    tag_names=['display_id', 'ad_id']))\
.saveAsTextFile(fname)

Deleted /user/outbrain/vw/train.vw


In [64]:
fname = '/user/outbrain/vw/val.vw' # _randomSplit
! hdfs dfs -rm -r $fname
val_data.rdd\
.map(lambda x: convert_to_vw(
    x, feature_space=feature_space, # float_space=float_space, 
    tag_names=['display_id', 'ad_id']))\
.saveAsTextFile(fname)

Deleted /user/outbrain/vw/val.vw


In [65]:
fname = '/user/outbrain/vw/test_28032022.vw'
! hdfs dfs -rm -r $fname
test_data.rdd\
.map(lambda x: convert_to_vw(
    x, task='test', feature_space=feature_space, # float_space=float_space, 
    tag_names=['display_id', 'ad_id']))\
.saveAsTextFile(fname)

rm: `/user/outbrain/vw/test_28032022.vw': No such file or directory


## Обучение Vowpal Wabbit

In [66]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

for task_type in ['train', 'test']:
    if not os.path.exists(f'/home/ubuntu/storage/outbrain/vw/{task_type}'):
        os.makedirs(f'/home/ubuntu/storage/outbrain/vw/{task_type}')

In [32]:
'''
! sudo wget http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624 -O /usr/bin/vw
! sudo chmod +x /usr/bin/vw
! sudo chown ubuntu /usr/bin/vw
! sudo apt-get update -y && sudo apt-get install graphviz -y
! pip install numpy pandas sklearn dateparser pandarallel ipywidgets catboost graphviz
! /opt/conda/bin/jupyter nbextension enable --py widgetsnbextension
'''

'\n! sudo wget http://finance.yendor.com/ML/VW/Binaries/vw-8.20190624 -O /usr/bin/vw\n! sudo chmod +x /usr/bin/vw\n! sudo chown ubuntu /usr/bin/vw\n! sudo apt-get update -y && sudo apt-get install graphviz -y\n! pip install numpy pandas sklearn dateparser pandarallel ipywidgets catboost graphviz\n! /opt/conda/bin/jupyter nbextension enable --py widgetsnbextension\n'

In [67]:
# Проверяем, что vw работает
! vw --help | head

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1
driver:
  --onethread           Disable parse thread
VW options:
  --ring_size arg (=256, ) size of example ring
  --strict_parse           throw on malformed examples
Update options:
  -l [ --learning_rate ] arg Set learning rate
  --power_t arg              t power value
  --decay_learning_rate arg  Set Decay factor for learning_rate between passes
  --initial_t arg            initial t value


In [68]:
# ! hdfs dfs -cat '/user/outbrain/vw/train_weighted.vw/*' > /home/ubuntu/storage/outbrain/vw/train/train_weighted.vw
# ! hdfs dfs -cat '/user/outbrain/vw/train.vw/*' > /home/ubuntu/storage/outbrain/vw/train/train.vw 
# ! hdfs dfs -cat '/user/outbrain/vw/train_randomSplit.vw/*' > /home/ubuntu/storage/outbrain/vw/train/train_randomSplit.vw
# ! hdfs dfs -cat '/user/outbrain/vw/val_randomSplit.vw/*' > /home/ubuntu/storage/outbrain/vw/train/val_randomSplit.vw
! hdfs dfs -cat '/user/outbrain/vw/train.vw/*' > /home/ubuntu/storage/outbrain/vw/train/train_28032022.vw
! hdfs dfs -cat '/user/outbrain/vw/val.vw/*' > /home/ubuntu/storage/outbrain/vw/train/val_28032022.vw

In [69]:
! cat /home/ubuntu/storage/outbrain/vw/train/train_28032022.vw | head -n 5

-1 display_id_3195987_ad_id_148|a 148 |b 1773858 |c 1 |d 1318500090 |e -1077041708 |f -912215920 |g 602 |h 1515 1506 |i -564571673 -2087840067 |j 872 |k 26 |l -1 |m -1 |n 93 285 276 174 250 117 |o 471520 |p 34 |q 19 |r 1515 1512 |s 0 |t 2644 |u 113 |v 5 |w 5 |x 93 285 296 292
-1 display_id_3262691_ad_id_148|a 148 |b 1773858 |c 1 |d 354655130 |e -1077041708 |f 410653915 |g 508 |h 1515 1506 |i -564571673 -2087840067 |j 872 |k 26 |l -1 |m -1 |n 93 285 276 174 250 117 |o 471520 |p 34 |q 19 |r 1515 1512 |s 0 |t 2644 |u 113 |v 5 |w 5 |x 93 285 296 292
-1 display_id_3267986_ad_id_148|a 148 |b 348758 |c 1 |d -441949987 |e -1077041708 |f 1886535265 |g 803 |h 1515 1510 |i -1475663123 -2027717077 |j 2760 |k 939 |l -1 |m -1 |n 294 117 129 296 62 244 291 285 170 163 |o 471520 |p 34 |q 19 |r 1515 1512 |s 0 |t 2644 |u 113 |v 5 |w 5 |x 93 285 296 292
-1 display_id_3678937_ad_id_148|a 148 |b 1834297 |c 1 |d 1318500090 |e -1077041708 |f -912215920 |g 602 |h 1515 1612 |i -1120691283 1939100908 |j 6862 |k

In [70]:
! cat /home/ubuntu/storage/outbrain/vw/train/val_28032022.vw | head -n 5

1 display_id_13905068_ad_id_496|a 496 |b 1025141 |c 3 |d -1239361328 |e -1077041708 |f 1054775907 |g 542 |h 1511 1503 |i 1328232759 -589023711 558851960 -1413770349 |j 6862 |k 637 |l 2 |m 4 |n 292 93 |o 78703 |p 83 |q 76 |r 1513 1514 |s 0 |t 3183 |u 139 |v -1 |w -1 |x 0
-1 display_id_130971_ad_id_1238|a 1238 |b 142452 |c 3 |d 137590007 |e -1471120248 |f 936913235 |g 0 |h 1603 1515 |i 0 |j 1907 |k 284 |l -1 |m -1 |n 143 193 71 75 |o 185913 |p 114 |q 67 |r 2006 2002 |s 1312092024 -1434132067 |t 2633 |u -1 |v 5 |w 12 |x 75 143 16 147 102
-1 display_id_597353_ad_id_1238|a 1238 |b 1653825 |c 2 |d 1851824643 |e -911176333 |f -333116310 |g 0 |h 1603 1100 |i 0 |j 10030 |k 1089 |l 3 |m 12 |n 16 179 |o 185913 |p 114 |q 67 |r 2006 2002 |s 1312092024 -1434132067 |t 2633 |u -1 |v 5 |w 12 |x 75 143 16 147 102
-1 display_id_23542_ad_id_1580|a 1580 |b 964380 |c 1 |d 444233572 |e -1077041708 |f -1008012134 |g 722 |h 1514 1510 |i 0 |j 7031 |k 637 |l 6 |m 18 |n 292 24 238 258 207 93 |o 29858 |p 292 |q 39

In [79]:
from itertools import combinations
import string

def make_interactions(n_feat=10, comb=2):
    
    feat = string.ascii_letters[:n_feat]
    return ' '.join(map(lambda x: ''.join(x), combinations(feat, comb)))

interactions = make_interactions(len(feature_space), 2)
interactions

'ab ac ad ae af ag ah ai aj ak al am an ao ap aq ar as at au av aw ax bc bd be bf bg bh bi bj bk bl bm bn bo bp bq br bs bt bu bv bw bx cd ce cf cg ch ci cj ck cl cm cn co cp cq cr cs ct cu cv cw cx de df dg dh di dj dk dl dm dn do dp dq dr ds dt du dv dw dx ef eg eh ei ej ek el em en eo ep eq er es et eu ev ew ex fg fh fi fj fk fl fm fn fo fp fq fr fs ft fu fv fw fx gh gi gj gk gl gm gn go gp gq gr gs gt gu gv gw gx hi hj hk hl hm hn ho hp hq hr hs ht hu hv hw hx ij ik il im in io ip iq ir is it iu iv iw ix jk jl jm jn jo jp jq jr js jt ju jv jw jx kl km kn ko kp kq kr ks kt ku kv kw kx lm ln lo lp lq lr ls lt lu lv lw lx mn mo mp mq mr ms mt mu mv mw mx no np nq nr ns nt nu nv nw nx op oq or os ot ou ov ow ox pq pr ps pt pu pv pw px qr qs qt qu qv qw qx rs rt ru rv rw rx st su sv sw sx tu tv tw tx uv uw ux vw vx wx'

In [80]:
%%time
! vw /home/ubuntu/storage/outbrain/vw/train/train_28032022.vw \
    --loss_function logistic \
    --final_regressor /home/ubuntu/storage/outbrain/vw/wide_ftrl.model.bin \
    --initial_regressor /home/ubuntu/storage/outbrain/vw/wide_ftrl.model.bin \
    --interactions $interactions \
    --passes 1 --bit_precision 24 --learning_rate 0.05 \
    --ftrl --ftrl_alpha 0.05 --ftrl_beta 0.5 --l1 1 \
    --cache /home/ubuntu/storage/outbrain/vw/train/train_28032022.vw.cache

creating features for following interactions: ab ac ad ae af ag ah ai aj ak al am an ao ap aq ar as at au av aw ax bc bd be bf bg bh bi bj bk bl bm bn bo bp bq br bs bt bu bv bw bx cd ce cf cg ch ci cj ck cl cm cn co cp cq cr cs ct cu cv cw cx de df dg dh di dj dk dl dm dn do dp dq dr ds dt du dv dw dx ef eg eh ei ej ek el em en eo ep eq er es et eu ev ew ex fg fh fi fj fk fl fm fn fo fp fq fr fs ft fu fv fw fx gh gi gj gk gl gm gn go gp gq gr gs gt gu gv gw gx hi hj hk hl hm hn ho hp hq hr hs ht hu hv hw hx ij ik il im in io ip iq ir is it iu iv iw ix jk jl jm jn jo jp jq jr js jt ju jv jw jx kl km kn ko kp kq kr ks kt ku kv kw kx lm ln lo lp lq lr ls lt lu lv lw lx mn mo mp mq mr ms mt mu mv mw mx no np nq nr ns nt nu nv nw nx op oq or os ot ou ov ow ox pq pr ps pt pu pv pw px qr qs qt qu qv qw qx rs rt ru rv rw rx st su sv sw sx tu tv tw tx uv uw ux vw vx wx 
using l1 regularization = 1
final_regressor = /home/ubuntu/storage/outbrain/vw/wide_ftrl.model.bin
Enabling FTRL based optimi

In [73]:
%%time
if not os.path.exists('/home/ubuntu/storage/outbrain/predictions'):
    os.makedirs('/home/ubuntu/storage/outbrain/predictions')

! vw --testonly --link=logistic \
    --initial_regressor /home/ubuntu/storage/outbrain/vw/wide_ftrl.model.bin \
    --predictions predictions/val_wide_ftrl.txt \
    /home/ubuntu/storage/outbrain/vw/train/val_28032022.vw

only testing
predictions = predictions/val_wide_ftrl.txt
Enabling FTRL based optimization
Algorithm used: Proximal-FTRL
ftrl_alpha = 0.005
ftrl_beta = 0.1
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = /home/ubuntu/storage/outbrain/vw/train/val_28032022.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
13.905986 13.905986            1            1.0   1.0000   0.0613       31
8.777140 3.648294            2            2.0  -1.0000   0.0517       35
5.417743 2.058347            4            4.0  -1.0000   0.0620       32
3.600470 1.783198            8            8.0  -1.0000   0.1251       37
2.504075 1.407680           16           16.0  -1.0000   0.1916       29
1.615840 0.727605           32           32.0  -1.0000   0.2661       32
2.519887 3.423934           64           64.0   1.0000   0.2535       49
2.474804 2.4

In [74]:
def mean_average_precision_ak(user_id, y_expected, y_pred, k=12):
    mask = y_expected == 1
    users = np.unique(user_id[mask])
    total = 0
    for i, user in enumerate(users):
        user_pred = y_pred[mask & (user_id == user)][:k]
        arange = np.arange(len(user_pred)) + 1
        total += np.sum(user_pred / arange)
        if i % 100000 == 0:
            print(i)
    
    return total / len(users)

def calc_metrics(predictions_filename, answers_filename, map_k=None):
    def read_target_from_vw(vw_record, ):
        out = {'display_id': int(vw_record.split('_')[2]),
               'ad_id':      int(vw_record.split('|')[0].split('_')[-1])}
        
        out['clicked'] = float(vw_record.split(' ')[0])
        
        return out
    
    y_pred = {'clicked': [], 'display_id': [], 'ad_id': []}
    with open(predictions_filename, 'r') as f:
        print('Reading predictions...')
        for i, line in enumerate(f):
            for k, v in read_target_from_vw(line).items():
                y_pred[k].append(v)
            if i % 1000000 == 0:
                print(f'Current prediction id: {i}')       
    y_pred = pd.DataFrame(y_pred)
        
    y_true = {'clicked': [], 'display_id': [], 'ad_id': []}
    with open(answers_filename, 'r') as f:
        print('Reading answers...')
        for i, line in enumerate(f):
            for k, v in read_target_from_vw(line).items():
                y_true[k].append(v)
            if i % 1000000 == 0:
                print(f'Current answer id: {i}')  
    y_true = pd.DataFrame(y_true)
    
    if map_k is not None:
        # Тут что-то похожее на метрику, но вроде как не то
        predictions = pd.merge(y_true, y_pred, on=['display_id', 'ad_id'], suffixes=('_true', '_pred'))
        prak = predictions[predictions.clicked_true == 1].groupby('display_id')['clicked_pred']\
        .agg(lambda x: np.sum(((np.array(x)[:map_k] + 1) / 2) / (np.arange(len(np.array(x)[:map_k])) + 1)))
        mapak = prak.sum() / len(prak)
    else:
        mapak = 0
    
    roc_auc = roc_auc_score(y_true.clicked, y_pred.clicked)
    
    print('roc_auc: {:.3f} mean_average_precision_a{}: {:.3f}'\
         .format(roc_auc, map_k, mapak))
    
    return y_pred, y_true, (roc_auc, mapak)

In [75]:
%%time
y_pred, y_true, metrics = calc_metrics(
    '/home/ubuntu/storage/outbrain/predictions/val_wide_ftrl.txt',
    '/home/ubuntu/storage/outbrain/vw/train/val_28032022.vw',
    map_k=12
)

Reading predictions...
Current prediction id: 0
Current prediction id: 1000000
Current prediction id: 2000000
Current prediction id: 3000000
Current prediction id: 4000000
Current prediction id: 5000000
Current prediction id: 6000000
Current prediction id: 7000000
Current prediction id: 8000000
Reading answers...
Current answer id: 0
Current answer id: 1000000
Current answer id: 2000000
Current answer id: 3000000
Current answer id: 4000000
Current answer id: 5000000
Current answer id: 6000000
Current answer id: 7000000
Current answer id: 8000000
roc_auc: 0.733 mean_average_precision_a12: 0.645
CPU times: user 2min 21s, sys: 4.45 s, total: 2min 25s
Wall time: 2min 25s


## Submission

In [76]:
! hdfs dfs -cat '/user/outbrain/vw/test_28032022.vw/*' > /home/ubuntu/storage/outbrain/vw/test/test_28032022.vw

In [81]:
%%time
! vw --testonly --link=logistic \
    --initial_regressor /home/ubuntu/storage/outbrain/vw/wide_ftrl.model.bin \
    --predictions predictions/wide_interactions_ftrl_submission.txt \
    /home/ubuntu/storage/outbrain/vw/test/test_28032022.vw

creating features for following interactions: ab ac ad ae af ag ah ai aj ak al am an ao ap aq ar as at au av aw ax bc bd be bf bg bh bi bj bk bl bm bn bo bp bq br bs bt bu bv bw bx cd ce cf cg ch ci cj ck cl cm cn co cp cq cr cs ct cu cv cw cx de df dg dh di dj dk dl dm dn do dp dq dr ds dt du dv dw dx ef eg eh ei ej ek el em en eo ep eq er es et eu ev ew ex fg fh fi fj fk fl fm fn fo fp fq fr fs ft fu fv fw fx gh gi gj gk gl gm gn go gp gq gr gs gt gu gv gw gx hi hj hk hl hm hn ho hp hq hr hs ht hu hv hw hx ij ik il im in io ip iq ir is it iu iv iw ix jk jl jm jn jo jp jq jr js jt ju jv jw jx kl km kn ko kp kq kr ks kt ku kv kw kx lm ln lo lp lq lr ls lt lu lv lw lx mn mo mp mq mr ms mt mu mv mw mx no np nq nr ns nt nu nv nw nx op oq or os ot ou ov ow ox pq pr ps pt pu pv pw px qr qs qt qu qv qw qx rs rt ru rv rw rx st su sv sw sx tu tv tw tx uv uw ux vw vx wx 
only testing
predictions = predictions/wide_interactions_ftrl_submission.txt
Enabling FTRL based optimization
Algorithm used:

In [82]:
%%time
if not os.path.exists('/home/ubuntu/storage/outbrain/submissions'):
    os.makedirs('/home/ubuntu/storage/outbrain/submissions')

predictions_filename = 'wide_interactions_ftrl_submission'
submission_filename = 'wide_interactions_ftrl_submission'
prob = .10 # cutoff

def submission_mapper(data):
    ads = ' '.join(np.array(data.ad_id)[
        # np.array(data.clicked) >= min(max(np.array(data.clicked)), prob) # Выдать хотя бы 1 предикт
        np.argsort(-np.array(data.clicked)) # no need to cutoff actually -_-
    ].astype(str))
    template = '{},{}'.format(data.display_id, ads)
    return template

def save_submission_to_parquet(
    predictions_filename,
):    
    def read_target_from_vw(vw_record):
        out = {'display_id': int(vw_record.split('_')[2]),
               'ad_id':      int(vw_record.split('_')[-1]),
               'clicked':    float(vw_record.split(' ')[0])}
        return out
    
    submit = {'clicked': [], 'display_id': [], 'ad_id': []}
    with open(predictions_filename, 'r') as f:
        print('Reading predictions...')
        for i, line in enumerate(f):
            for k, v in read_target_from_vw(line).items():
                submit[k].append(v)
            if i % 10000000 == 0:
                print(f'Current prediction id: {i}')
                
    submit = pd.DataFrame(submit)
    print('Saving parquet...')
    submit.to_parquet(predictions_filename.replace('txt', 'parquet'))
    return submit


! hdfs dfs -mkdir -p /user/outbrain/predictions/
! hdfs dfs -mkdir -p /user/outbrain/submissions/
submit = save_submission_to_parquet(f'predictions/{predictions_filename}.txt')

print('Putting parquet into hdfs...')
hdfs_path = f'/user/outbrain/predictions/{predictions_filename}.parquet'
f_path = f'/home/ubuntu/storage/outbrain/predictions/{predictions_filename}.parquet'
! hdfs dfs -rm -r $hdfs_path
! hdfs dfs -put\
    $f_path\
    /user/outbrain/predictions/

print(f'Making submission file for model: {submission_filename}...')
_submit = se.read.parquet(f'/user/outbrain/predictions/{predictions_filename}.parquet')
_submit = _submit.groupBy('display_id')\
.agg(F.collect_list('ad_id').alias('ad_id'),
     F.collect_list('clicked').alias('clicked'))

header = sc.parallelize(["display_id,ad_id"])
rdd = _submit.rdd.map(submission_mapper)

hdfs_path = f'/user/outbrain/submissions/{submission_filename}.csv'
! hdfs dfs -rm -r $hdfs_path
header.union(rdd).saveAsTextFile(f'/user/outbrain/submissions/{submission_filename}.csv')

hdfs_path = f'/user/outbrain/submissions/{submission_filename}.csv/*'
f_path = f'/home/ubuntu/storage/outbrain/submissions/{submission_filename}.csv'

! rm -r $f_path
! hdfs dfs -cat $hdfs_path > $f_path

print('Submission file path:\n'+f_path)
print('Kagge command for submitting:')
print(
    f'kaggle competitions submit -f {f_path} -m',
    f'\'model: {predictions_filename}\' outbrain-click-prediction'
)
print('Head of submission file:')
! cat $f_path | head -n 5

Reading predictions...
Current prediction id: 0
Current prediction id: 10000000
Current prediction id: 20000000
Current prediction id: 30000000
Saving parquet...
Putting parquet into hdfs...
rm: `/user/outbrain/predictions/wide_interactions_ftrl_submission.parquet': No such file or directory
Making submission file for model: wide_interactions_ftrl_submission...
rm: `/user/outbrain/submissions/wide_interactions_ftrl_submission.csv': No such file or directory


rm: cannot remove '/home/ubuntu/storage/outbrain/submissions/wide_interactions_ftrl_submission.csv': No such file or directory
Submission file path:
/home/ubuntu/storage/outbrain/submissions/wide_interactions_ftrl_submission.csv
Kagge command for submitting:
kaggle competitions submit -f /home/ubuntu/storage/outbrain/submissions/wide_interactions_ftrl_submission.csv -m 'model: wide_interactions_ftrl_submission' outbrain-click-prediction
Head of submission file:
display_id,ad_id
16874671,220854 99542 78301 87000 78274 269017
16874715,169925 93787 285834 148135
16875905,8117 95155 257550 24621 171403 223796
16876343,322537 293511 215846
cat: write error: Broken pipe
CPU times: user 1min 10s, sys: 6.32 s, total: 1min 16s
Wall time: 3min 43s


## Итоги

Сабмишны:

- Фичи из [бейслайна](http://dsnotes.com/post/2017-01-27-lessons-learned-from-outbrain-click-prediction-kaggle-competition/) (promoted_content + events, 10 штук) + interactions 2: 
    - Скрин: https://storage.yandexcloud.net/nsrazuvaevbucket/baseline.png
    - Файл: https://storage.yandexcloud.net/nsrazuvaevbucket/baseline_ftrl_submission.csv
    - MAP@12: .651, AUC: 0,737

- Фичи из бейслайна + остальные таблицы (+ documents_categories + documents_entities + documents_meta + documents_topics; 24 фичи в сумме), кроме page_views:
    - Скрин: https://storage.yandexcloud.net/nsrazuvaevbucket/wide.png
    - Файл: https://storage.yandexcloud.net/nsrazuvaevbucket/wide_ftrl_submission.csv
    - MAP@12: .644, AUC: 0,733

- Вариант выше + interactions 2 дает качество еще хуже:
    - Скрин: https://storage.yandexcloud.net/nsrazuvaevbucket/wide_interactions.png
    - Файл: https://storage.yandexcloud.net/nsrazuvaevbucket/wide_interactions_ftrl_submission.csv

> Либо добавленные фичи бесполезны, либо я не умею подбирать параметры -_-